<a href="https://colab.research.google.com/github/agam844/Active-Learning/blob/main/Bank_Active.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip --quiet install openprompt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openprompt.data_utils import InputExample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [3]:
np.random.seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = '/content/drive/MyDrive/bank-full.csv'
df = pd.read_csv(file_path)

In [6]:
df = df[(df['job'] != 'unknown') & (df['education'] != 'unknown') & (df['contact'] != 'unknown') & (df['poutcome'] != 'unknown') ]

In [7]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
24060,33,admin.,married,tertiary,no,882,no,no,telephone,21,oct,39,1,151,3,failure,no
24062,42,admin.,single,secondary,no,-247,yes,yes,telephone,21,oct,519,1,166,1,other,yes
24064,33,services,married,secondary,no,3444,yes,no,telephone,21,oct,144,1,91,4,failure,yes
24072,36,management,married,tertiary,no,2415,yes,no,telephone,22,oct,73,1,86,4,other,no
24077,36,management,married,tertiary,no,0,yes,no,telephone,23,oct,140,1,143,3,failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45199,34,blue-collar,single,secondary,no,1475,yes,no,cellular,16,nov,1166,3,530,12,other,no
45201,53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success,yes
45204,73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes


In [8]:
# Encode categorical variables using one-hot encoding
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
data_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [9]:
data_encoded
data_encoded = data_encoded.reset_index(drop=True)

In [10]:
data_encoded

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,y_yes
0,33,882,21,39,1,151,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,42,-247,21,519,1,166,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,33,3444,21,144,1,91,4,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,36,2415,22,73,1,86,4,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,36,0,23,140,1,143,3,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7837,34,1475,16,1166,3,530,12,1,0,0,...,0,0,0,0,1,0,0,1,0,0
7838,53,583,17,226,1,184,4,0,0,0,...,0,0,0,0,1,0,0,0,1,1
7839,73,2850,17,300,1,40,8,0,0,0,...,0,0,0,0,1,0,0,0,0,1
7840,72,5715,17,1127,5,184,3,0,0,0,...,0,0,0,0,1,0,0,0,1,1


In [11]:
x = data_encoded.drop('y_yes', axis=1)
y = data_encoded['y_yes']

In [12]:
# Split the data into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
# Create a logistic regression model
model = LogisticRegression(max_iter=100)

# Train the model on the training dataset
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
# Make predictions on the testing dataset
y_pred = model.predict(x_test)
# Get the probability estimates for each prediction
y_pred_prob = model.predict_proba(x_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8266411727214786


## Active learning

Low confidence and wrong solution

In [15]:
# Scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [16]:
num_entries = min(600, len(x_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [17]:
wrong = dict()
correct = dict()
merged_dict = dict()

#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  wrong = dict()
  correct = dict()

  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  # for i in range(len(remaining_x_train)):
  #   if y_pred[i] != remaining_y_train[i]:
  #     # np.max(y_pred_proba, axis=1)[i] = 0

  for i in range(len(remaining_x_train)):
    if y_pred[i] != remaining_y_train[i]:
      wrong.update({max(y_pred_proba[i]):i})
    else:
      correct.update({max(y_pred_proba[i]):i})

  wrong = dict(sorted(wrong.items()))
  correct = dict(sorted(correct.items()))

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 100

  if len(wrong) < num_additional_samples:
    num = num_additional_samples - len(wrong)
    additional_samples = {k: correct[k] for k in list(correct)[:num]}
    wrong.update(additional_samples)

  values = [value for value in wrong.values()]

  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Iteration: 1
Number of labeled samples: 700
Number of remaining samples: 5573
Iteration: 2
Number of labeled samples: 800
Number of remaining samples: 5473
Iteration: 3
Number of labeled samples: 900
Number of remaining samples: 5373
Iteration: 4
Number of labeled samples: 1000
Number of remaining samples: 5273
Iteration: 5
Number of labeled samples: 1100
Number of remaining samples: 5173
Iteration: 6
Number of labeled samples: 1200
Number of remaining samples: 5073
Iteration: 7
Number of labeled samples: 1300
Number of remaining samples: 4973
Iteration: 8
Number of labeled samples: 1400
Number of remaining samples: 4873
Iteration: 9
Number of labeled samples: 1500
Number of remaining samples: 4773
Iteration: 10
Number of labeled samples: 1600
Number of remaining samples: 4673


In [18]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

In [19]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9400813182109994


High Confidence and Wrong Labels

In [25]:
num_entries = min(600, len(x_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [26]:
wrong = dict()
correct = dict()
merged_dict = dict()

#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  wrong = dict()
  correct = dict()

  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  # for i in range(len(remaining_x_train)):
  #   if y_pred[i] != remaining_y_train[i]:
  #     # np.max(y_pred_proba, axis=1)[i] = 0

  for i in range(len(remaining_x_train)):
    if y_pred[i] != remaining_y_train[i]:
      wrong.update({max(y_pred_proba[i]):i})
    else:
      correct.update({max(y_pred_proba[i]):i})

  wrong = dict(sorted(wrong.items(), reverse=True))
  correct = dict(sorted(correct.items(), reverse=True))

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 100

  if len(wrong) < num_additional_samples:
    num = num_additional_samples - len(wrong)
    additional_samples = {k: correct[k] for k in list(correct)[:num]}
    wrong.update(additional_samples)

  values = [value for value in wrong.values()]

  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Iteration: 1
Number of labeled samples: 700
Number of remaining samples: 5573
Iteration: 2
Number of labeled samples: 800
Number of remaining samples: 5473
Iteration: 3
Number of labeled samples: 900
Number of remaining samples: 5373
Iteration: 4
Number of labeled samples: 1000
Number of remaining samples: 5273
Iteration: 5
Number of labeled samples: 1100
Number of remaining samples: 5173
Iteration: 6
Number of labeled samples: 1200
Number of remaining samples: 5073
Iteration: 7
Number of labeled samples: 1300
Number of remaining samples: 4973
Iteration: 8
Number of labeled samples: 1400
Number of remaining samples: 4873
Iteration: 9
Number of labeled samples: 1500
Number of remaining samples: 4773
Iteration: 10
Number of labeled samples: 1600
Number of remaining samples: 4673


In [27]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

In [28]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9465011769741065


Low confidence and Correct Labels

In [29]:
num_entries = min(600, len(x_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [30]:
wrong = dict()
correct = dict()
merged_dict = dict()

#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  wrong = dict()
  correct = dict()

  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  # for i in range(len(remaining_x_train)):
  #   if y_pred[i] != remaining_y_train[i]:
  #     # np.max(y_pred_proba, axis=1)[i] = 0

  for i in range(len(remaining_x_train)):
    if y_pred[i] != remaining_y_train[i]:
      wrong.update({max(y_pred_proba[i]):i})
    else:
      correct.update({max(y_pred_proba[i]):i})

  wrong = dict(sorted(wrong.items()))
  correct = dict(sorted(correct.items()))

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 100

  if len(correct) < num_additional_samples:
    num = num_additional_samples - len(correct)
    additional_samples = {k: wrong[k] for k in list(wrong)[:num]}
    correct.update(additional_samples)

  values = [value for value in correct.values()]

  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Iteration: 1
Number of labeled samples: 700
Number of remaining samples: 5573
Iteration: 2
Number of labeled samples: 800
Number of remaining samples: 5473
Iteration: 3
Number of labeled samples: 900
Number of remaining samples: 5373
Iteration: 4
Number of labeled samples: 1000
Number of remaining samples: 5273
Iteration: 5
Number of labeled samples: 1100
Number of remaining samples: 5173
Iteration: 6
Number of labeled samples: 1200
Number of remaining samples: 5073
Iteration: 7
Number of labeled samples: 1300
Number of remaining samples: 4973
Iteration: 8
Number of labeled samples: 1400
Number of remaining samples: 4873
Iteration: 9
Number of labeled samples: 1500
Number of remaining samples: 4773
Iteration: 10
Number of labeled samples: 1600
Number of remaining samples: 4673


In [31]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

In [32]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.827305799272416


High Confidence and Correct Labels

In [33]:
num_entries = min(600, len(x_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

In [34]:
wrong = dict()
correct = dict()
merged_dict = dict()

#active learning loop
num_iterations = 10

for iteration in range(num_iterations):
  wrong = dict()
  correct = dict()

  # Initialize the logistic regression model
  model = LogisticRegression(max_iter=100)
  model.fit(selected_x_train, selected_y_train)

  y_pred = model.predict(remaining_x_train)
  y_pred_proba = model.predict_proba(remaining_x_train)

  # for i in range(len(remaining_x_train)):
  #   if y_pred[i] != remaining_y_train[i]:
  #     # np.max(y_pred_proba, axis=1)[i] = 0

  for i in range(len(remaining_x_train)):
    if y_pred[i] != remaining_y_train[i]:
      wrong.update({max(y_pred_proba[i]):i})
    else:
      correct.update({max(y_pred_proba[i]):i})

  wrong = dict(sorted(wrong.items(), reverse=True))
  correct = dict(sorted(correct.items(), reverse=True))

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 100

  if len(correct) < num_additional_samples:
    num = num_additional_samples - len(correct)
    additional_samples = {k: wrong[k] for k in list(wrong)[:num]}
    correct.update(additional_samples)

  values = [value for value in correct.values()]

  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Iteration: 1
Number of labeled samples: 700
Number of remaining samples: 5573
Iteration: 2
Number of labeled samples: 800
Number of remaining samples: 5473
Iteration: 3
Number of labeled samples: 900
Number of remaining samples: 5373
Iteration: 4
Number of labeled samples: 1000
Number of remaining samples: 5273
Iteration: 5
Number of labeled samples: 1100
Number of remaining samples: 5173
Iteration: 6
Number of labeled samples: 1200
Number of remaining samples: 5073
Iteration: 7
Number of labeled samples: 1300
Number of remaining samples: 4973
Iteration: 8
Number of labeled samples: 1400
Number of remaining samples: 4873
Iteration: 9
Number of labeled samples: 1500
Number of remaining samples: 4773
Iteration: 10
Number of labeled samples: 1600
Number of remaining samples: 4673


In [35]:
model = LogisticRegression(max_iter=100)
model.fit(selected_x_train, selected_y_train)
y_pred = model.predict(remaining_x_train)

In [36]:
accuracy = accuracy_score(remaining_y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.788144660817462
